Chargement des Librairies

In [1]:
#Librairies
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
from sklearn.cluster import AgglomerativeClustering, KMeans

Chargement des CSV

In [2]:
#GenomeScores qui donne la relevance par tagIg et par movieId

dfgenome=pd.read_csv("../data/genome-scores.csv" )
display(dfgenome.head(5))
display(dfgenome.info())

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11709768 entries, 0 to 11709767
Data columns (total 3 columns):
 #   Column     Dtype  
---  ------     -----  
 0   movieId    int64  
 1   tagId      int64  
 2   relevance  float64
dtypes: float64(1), int64(2)
memory usage: 268.0 MB


None

In [3]:
#GenomeTags qui donne la correspondance des Tags avec leur TagId
dftags=pd.read_csv("../data/genome-tags.csv", index_col="tagId")
display(dftags.head(5))
display(dftags.info())

,tag
tagId,
1,007
2,007 (series)
3,18th century
4,1920s
5,1930s


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1128 entries, 1 to 1128
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tag     1128 non-null   object
dtypes: object(1)
memory usage: 17.6+ KB


None

In [4]:
#movies  qui donne la correspondance des titre des films, leur genre avec leur movieId
dfmovies=pd.read_csv("../data/movies.csv", sep=',', index_col="movieId")
display(dfmovies.info())
display(dfmovies.head(5))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27278 entries, 1 to 131262
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   27278 non-null  object
 1   genres  27278 non-null  object
dtypes: object(2)
memory usage: 639.3+ KB


None

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [5]:
#ratings  qui recence les notes des utilisateurs avec la correspondance des Id des utilisateurs, des Id des films,
#la note (rating) et l'instant de la notation (timestamp)

dfratings=pd.read_csv("../data/ratings.csv", sep=',')
display(dfratings.info())
display(dfratings.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 610.4 MB


None

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [6]:
#groupby tagId and sum to compute the total relevance by tagId. Desascending sorting to keep only the 50 most relevant. 

toprelevance = dfgenome.groupby('tagId').agg(sum).sort_values('relevance',ascending=False).head(50).index


In [7]:
#selection in the main dataframe dfgenome from the grouped dataframe selection above.

genome_top = dfgenome[dfgenome['tagId'].isin(toprelevance)]

In [8]:
#check of the tab from the selection

display(genome_top.head(5))
display(genome_top.info())

,movieId,tagId,relevance
20,1,21,0.33025
21,1,22,0.28250
168,1,169,0.20525
187,1,188,0.48600
194,1,195,0.59000


<class 'pandas.core.frame.DataFrame'>
Int64Index: 519050 entries, 20 to 11709743
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   movieId    519050 non-null  int64  
 1   tagId      519050 non-null  int64  
 2   relevance  519050 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 15.8 MB


None

In [9]:
#compression of the selection table by pivot method.
genome_top=pd.pivot_table(genome_top, index="movieId",columns="tagId")

In [10]:
#check of the tab from the pivot method
display(genome_top.head(5))
display(genome_top.info())

relevance                                                       \
tagId        21       22       169     188      195      203      212    
movieId                                                                  
1         0.33025  0.28250  0.20525  0.4860  0.59000  0.92625  0.12350   
2         0.45100  0.60200  0.22825  0.4865  0.62600  0.87025  0.12975   
3         0.28000  0.20775  0.29950  0.4405  0.53800  0.34850  0.13750   
4         0.49875  0.44650  0.31100  0.4185  0.26125  0.24075  0.13425   
5         0.27575  0.21075  0.27050  0.5545  0.56800  0.39025  0.12300   

                                    ...                                     \
tagId       220      240      258   ...     972      982     992      999    
movieId                             ...                                      
1        0.78025  0.35550  0.28625  ...  0.75000  0.37550  0.4345  0.49000   
2        0.35625  0.11600  0.27550  ...  0.31650  0.18225  0.1260  0.44075   
3        0.18975  0.23050  0.23075  ...  0.28875  0.17350  0.4220  0.26675   
4        0.18825  0.13475  0.17200  ...  0.33050  0.17175  0.1985  0.18425   
5        0.18025  0.21200  0.17975  ...  0.27225  0.14450  0.3450  0.23850   

                                                               
tagId       1008     1051     1064     1070     1091     1104  
movieId                                                        
1        0.60950  0.36300  0.42050  0.26150  0.41050  0.29925  
2        0.14825  0.15050  0.22075  0.23475  0.19125  0.31600  
3        0.21175  0.20725  0.21875  0.41825  0.21250  0.16800  
4        0.17600  0.18325  0.28400  0.35950  0.18575  0.17850  
5        0.18300  0.15025  0.21400  0.28475  0.18750  0.26975  

[5 rows x 50 columns]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10381 entries, 1 to 131170
Data columns (total 50 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   (relevance, 21)    10381 non-null  float64
 1   (relevance, 22)    10381 non-null  float64
 2   (relevance, 169)   10381 non-null  float64
 3   (relevance, 188)   10381 non-null  float64
 4   (relevance, 195)   10381 non-null  float64
 5   (relevance, 203)   10381 non-null  float64
 6   (relevance, 212)   10381 non-null  float64
 7   (relevance, 220)   10381 non-null  float64
 8   (relevance, 240)   10381 non-null  float64
 9   (relevance, 258)   10381 non-null  float64
 10  (relevance, 270)   10381 non-null  float64
 11  (relevance, 277)   10381 non-null  float64
 12  (relevance, 299)   10381 non-null  float64
 13  (relevance, 302)   10381 non-null  float64
 14  (relevance, 323)   10381 non-null  float64
 15  (relevance, 351)   10381 non-null  float64
 16  (relevance, 378)   10

None

Clustering of the movies in function of their tag relevancy

In [11]:
clf = AgglomerativeClustering(n_clusters = 200)

clf.fit(genome_top)



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['tuple']. An error will be raised in 1.2.
  warnings.warn(


AgglomerativeClustering(n_clusters=200)

In [12]:
#Adding of the cluster labels to our Dataframe genome_top
genome_top["cluster"]=clf.labels_
    


Evaluation of the relevancy of the clustering by counting the number of 

occurence of each cluster


In [13]:
#counting the number of occurence of each cluster

cluster_repartition=genome_top.value_counts(subset="cluster")
print(cluster_repartition)

cluster
0      270
43     259
15     197
23     187
25     166
      ... 
172     17
197     17
167     16
146     15
130     14
Length: 200, dtype: int64
